In [1]:
import pandas as pd
import numpy as np
import psycopg2

# Connect to the newyork database
conn = psycopg2.connect(
    dbname = 'newyork',
    user = '',
    password = '',
    host = 'localhost',
    port = '5432'
)

# Get the top 50 nodes by negative price occurence
query = f"SELECT node, COUNT(*) FROM realtime_lbmp WHERE price < 0 GROUP BY node ORDER BY COUNT(*) DESC LIMIT 50;"
df = pd.read_sql_query(query, conn)
best_nodes = list(df.node)

# There are 58'943'742 elements in the file, so roughly 80k points per node
query = f"SELECT * FROM realtime_lbmp WHERE node IN %s"
df = pd.read_sql_query(query, conn, params=(tuple(best_nodes),))[['time','node','price']]
df = df.sort_values(by='time')
conn.close()

/var/folders/0x/l1m3sdhn7r59npk394yrpzk40000gn/T/ipykernel_6778/509204445.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/var/folders/0x/l1m3sdhn7r59npk394yrpzk40000gn/T/ipykernel_6778/509204445.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(best_nodes),))[['time','node','price']]


In [2]:
df['time'] = pd.to_datetime(df['time'])
df['hour'] = df['time'].dt.strftime('%Y-%m-%d %H')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df

,time,node,price,hour
2340193,2023-09-01 00:05:00,NM_ST_REGIS___HYD,22.20,2023-09-01 00
2340184,2023-09-01 00:05:00,MARBLE_RIVER_WT_PWR,22.75,2023-09-01 00
2340185,2023-09-01 00:05:00,MCINTYRE_115KV_TB2,22.68,2023-09-01 00
2340186,2023-09-01 00:05:00,MID___RAQUETTE_HYD,22.00,2023-09-01 00
2340188,2023-09-01 00:05:00,NEG NORTH_FLCN_SEA,23.02,2023-09-01 00
...,...,...,...,...
2339715,2024-06-01 00:00:00,JERICHO_RISE_WT_PWR,14.90,2024-06-01 00
2339716,2024-06-01 00:00:00,KNTFSRNC_46_KV_46KV_LOAD,15.21,2024-06-01 00
2339717,2024-06-01 00:00:00,LAWRNCAV_115KV_TB1,14.77,2024-06-01 00
2339706,2024-06-01 00:00:00,CLINTON_WT_PWR,15.12,2024-06-01 00


In [3]:
# Find the minimum length of prices
min_length = 1e9
for node in best_nodes:
    df_node = df[df.node==node]
    if len(df_node) < min_length:
        min_length = len(df_node)
print(f'The node with the least points has data for {min_length} time steps.')

# Set up as a dict and crop price lists at min_length
prices = {}
for node in best_nodes:

    # Get price by hour
    df2 = df[df.node==node][['hour','price']]
    hourly_avg = df2.groupby(['hour']).mean()

    prices[node] = [round(x,2) for x in list(hourly_avg.price)[:min_length]]

for key, value in prices.items():
    print(f'{key}: {value[:10]}...')

The node with the least points has data for 80226 time steps.
CLINTON_WT_PWR: [22.87, 20.28, 20.24, 19.43, 19.79, 22.73, 24.28, 23.25, 19.9, 18.58]...
CHATEAUG_WT_PWR: [22.87, 20.28, 20.24, 19.43, 19.79, 22.73, 24.28, 23.25, 19.9, 18.58]...
ELLENBURG_WT_PWR: [22.87, 20.28, 20.24, 19.43, 19.79, 22.73, 24.28, 23.25, 19.9, 18.58]...
JERICHO_RISE_WT_PWR: [22.8, 20.22, 20.18, 19.36, 19.72, 22.66, 24.2, 23.16, 19.84, 18.53]...
NORTH___COUNTRY_ESR: [22.8, 20.22, 20.18, 19.36, 19.72, 22.66, 24.2, 23.16, 19.84, 18.53]...
CHATEAUG_35_KV_LOAD: [22.88, 20.29, 20.24, 19.42, 19.78, 22.73, 24.28, 23.23, 19.89, 18.57]...
CHAT_HIGH_FALL_HYD: [22.88, 20.29, 20.24, 19.42, 19.78, 22.73, 24.28, 23.24, 19.89, 18.57]...
NEG NORTH_KES_CHATEGAY: [22.88, 20.29, 20.24, 19.42, 19.78, 22.73, 24.28, 23.23, 19.89, 18.57]...
NEG NORTH___ALICE_FALLS: [23.11, 20.48, 20.44, 19.64, 20.01, 22.97, 24.56, 23.56, 20.15, 18.78]...
NEG NORTH___LWR_SARANAC: [23.11, 20.48, 20.44, 19.64, 20.01, 22.97, 24.56, 23.56, 20.15, 18.78].

In [4]:
# Save the hourly data to CSV
df_for_csv = pd.DataFrame(prices)
df_for_csv.index = pd.to_datetime(list(hourly_avg.index))
df_for_csv.reset_index(inplace=True)
df_for_csv.rename(columns={'index': 'Time'}, inplace=True)
df_for_csv.to_csv('hourly_data.csv', index=False)